This Notebook gives a short introduction on how to do elementwise operations on CABLAB data cubes. First we load the CABLAB packages and get a reference to the air temperature cube

In [ ]:
using CABLAB
using CABLABPlots
plotlyjs()

In [ ]:
c = Cube("/home/jovyan/work/datacube/cablab-datacube-1.0.0/low-res/")

In [ ]:
tair = getCubeData(c,variable="air_temperature_2m")

In [ ]:
plotMAP(tair)

The air temperature in the data cube is given in Kelvin. Assume you prefer to work with degree Celsius instead, we can apply the map function on our cube. Here, the first argument is `t->t-273.15` is an anonymous function that takes the variable t, subtracts 273.15 and returns the result. The second argument is the cube our function is applied to element-wise. 

In [ ]:
tair_c = map(t->t-273.15, tair) 

Note that no computation is done yet on the cube, the function is applied only if the data is plotted, exported, or a call to `mapCube` is done. 

In [ ]:
plotMAP(tair_c)

As another example, let us try to estimate the (time-dependent) base respiration  $R_b$ under the bold assumption that $Q_{10} = 2$ is a global constant. If we assume the following model:

$$ R = R_b Q_{10}^{{T-T_{ref}} \over {10}}$$

where R is the ecosystem respiration and T the air temperature. Then 

$$ R_b = {{R} \over {Q_{10}^{{T-T_{ref}} \over {10}}}} $$

To do the calculation, we need two input data cubes, one for the temperature and one for the respiration. We define the function that should be applied:

In [ ]:
function rb_estimator(resp,t)
    Q10 = 2.0
    t_eff = Q10^((t-15.0)/10.0)
    return resp/t_eff
end

We load the respiration cube:

In [ ]:
resp = getCubeData(c,variable="terrestrial_ecosystem_respiration")

And we apply the map function again. Since our function rb_estimator takes two input arguments, we need to pass to cubes to the map function:

In [ ]:
rb = map(rb_estimator,resp,tair_c)

Now we can plot the results as maps and time series:

In [ ]:
plotMAP(rb,dmax = 5)

In [ ]:
plotTS(rb,lon=30.0,lat=50.0)

## Limitations

The `map` function here works only for functions that do elementwise operations on one or more input cubes. It is restricted to cases where all input cubes (and the output) have exactly the same dimensions, it will throw an error otherwise. 